In [55]:
import pandas as pd
import string
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import unicodedata
from sklearn.linear_model import RidgeCV
import numpy as np
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import shap
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from scikeras.wrappers import KerasRegressor
import xgboost as xgb
import tensorflow as tf
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [76]:
X_train = pd.read_csv('/Users/emilfalk/Desktop/X_train.csv')
X_test = pd.read_csv('/Users/emilfalk/Desktop/X_test.csv')
y_train = pd.read_csv('/Users/emilfalk/Desktop/y_train.csv')
y_test = pd.read_csv('/Users/emilfalk/Desktop/y_test.csv')

In [77]:
X_train = X_train.drop(columns=['Player']) 
X_test = X_test.drop(columns=['Player']) 

In [79]:
from itertools import combinations

non_basketball_features = ['years_with_team', 'draft_pos', 'all_star', 'num_teams', 'ig_followers', 'resigned', 'Agent']

# All combinations of 5 features
combinations_5 = list(combinations(non_basketball_features, 5))

# For each combination, get the 2 features not included
excluded_2_features = [list(set(non_basketball_features) - set(comb)) for comb in combinations_5]

In [80]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

## Multiple Linear Regression

In [69]:
mae_scores = []

for excluded in excluded_2_features:
    X_train_reduced = X_train.drop(columns=excluded)
    X_test_reduced = X_test.drop(columns=excluded)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_reduced) 
    X_test_scaled = scaler.transform(X_test_reduced)

    mlr = LinearRegression()
    mlr.fit(X_train_scaled, y_train.values.ravel())
    
    y_pred = mlr.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)

avg_mae = np.mean(mae_scores)
print(f"\nAverage MAE across all combinations: {avg_mae:.2f}")


Average MAE across all combinations: 3848627.03


## Ridge Regression

In [64]:
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

mae_scores = []

for excluded in excluded_2_features:
    X_train_reduced = X_train.drop(columns=excluded)
    X_test_reduced = X_test.drop(columns=excluded)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_reduced) 
    X_test_scaled = scaler.transform(X_test_reduced)

    ridge = Ridge(random_state = 42)
    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train.values.ravel())
    
    best_alpha = grid_search.best_params_['alpha']
    best_ridge = Ridge(alpha=best_alpha)
    best_ridge.fit(X_train_scaled, y_train.values.ravel())
    
    y_pred = best_ridge.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)

avg_mae = np.mean(mae_scores)
print(f"\nAverage MAE across all combinations: {avg_mae:.2f}")


Average MAE across all combinations: 3857485.04


## Lasso Regression

In [65]:
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

mae_scores = []

for excluded in excluded_2_features:
    X_train_reduced = X_train.drop(columns=excluded)
    X_test_reduced = X_test.drop(columns=excluded)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_reduced) 
    X_test_scaled = scaler.transform(X_test_reduced)

    lasso = Lasso(random_state = 42)
    grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train.values.ravel())
    
    best_alpha = grid_search.best_params_['alpha']
    best_lasso = Lasso(alpha=best_alpha)
    best_lasso.fit(X_train_scaled, y_train.values.ravel())
    
    y_pred = best_lasso.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)

avg_mae = np.mean(mae_scores)
print(f"\nAverage MAE across all combinations: {avg_mae:.2f}")

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.103e+15, tolerance: 2.703e+12
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.913e+15, tolerance: 2.756e+12
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, c


Average MAE across all combinations: 3799827.65


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.163e+15, tolerance: 2.977e+12
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.969e+15, tolerance: 3.181e+12
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, c

## Random Forest

In [84]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None]
}

mae_scores = []

for excluded in excluded_2_features:
    X_train = pd.read_csv('/Users/emilfalk/Desktop/X_train.csv')
    X_test = pd.read_csv('/Users/emilfalk/Desktop/X_test.csv')
    y_train = pd.read_csv('/Users/emilfalk/Desktop/y_train.csv')
    y_test = pd.read_csv('/Users/emilfalk/Desktop/y_test.csv')

    X_train = X_train.drop(columns=['Player']) 
    X_test = X_test.drop(columns=['Player'])
    
    X_train = X_train.drop(columns=excluded)
    X_test = X_test.drop(columns=excluded)

    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())

    best_rf = grid_search.best_estimator_
    best_rf.fit(X_train, y_train.values.ravel())

    y_pred = best_rf.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)

avg_mae = np.mean(mae_scores)
print(f"\nAverage MAE across all combinations: {avg_mae:.2f}")


Average MAE across all combinations: 3,033,319.24


## XGBoost

In [85]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1]
}

mae_scores = []

for excluded in excluded_2_features:
    X_train = pd.read_csv('/Users/emilfalk/Desktop/X_train.csv')
    X_test = pd.read_csv('/Users/emilfalk/Desktop/X_test.csv')
    y_train = pd.read_csv('/Users/emilfalk/Desktop/y_train.csv')
    y_test = pd.read_csv('/Users/emilfalk/Desktop/y_test.csv')

    X_train = X_train.drop(columns=['Player']) 
    X_test = X_test.drop(columns=['Player'])
    
    X_train = X_train.drop(columns=excluded)
    X_test = X_test.drop(columns=excluded)

    xgb = XGBRegressor(random_state=42, objective='reg:squarederror')
    grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())

    best_xgb = grid_search.best_estimator_
    best_xgb.fit(X_train, y_train.values.ravel())

    y_pred = best_xgb.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)

avg_mae = np.mean(mae_scores)
print(f"\nAverage MAE across all combinations: {avg_mae:.2f}")


Average MAE across all combinations: 3136236.58
